In [1]:
from os import path
import pandas as pd
import numpy as np
import json
from openai import OpenAI
from tqdm import tqdm
from dotenv import load_dotenv
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import os

DATA_DIR = r"c:\Users\Admin\Python\ABSA_Prompting\data"
RESULT_DIR = r"c:\Users\Admin\Python\ABSA_Prompting\results"
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RESULT_DIR, exist_ok=True)

c:\Users\Admin\Python\ABSA_Prompting\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# CONFIG
ViABSA_BP_dir = path.join(DATA_DIR, 'ViABSA_Restaurant')
test_file = path.join(ViABSA_BP_dir, 'data_test.csv')
test_df = pd.read_csv(test_file)

In [3]:
aspects = [
   "AMBIENCE#GENERAL",
   "DRINKS#PRICES",
   "DRINKS#QUALITY",
   "DRINKS#STYLE&OPTIONS",
   "FOOD#PRICES",
   "FOOD#QUALITY",
   "FOOD#STYLE&OPTIONS",
   "LOCATION#GENERAL",
   "RESTAURANT#GENERAL",
   "RESTAURANT#MISCELLANEOUS",
   "RESTAURANT#PRICES",
   "SERVICE#GENERAL"
]

In [4]:
sentiment_map = {
    1: "positive",
    2: "negative",
    3: "neutral"
}

def transform_aspect_sentiment(df, start=0, end=None):
    result = [] 
    
    if end is None:
        end = len(df)

    for idx, row in df.iloc[start:end].iterrows():
        entry = {
            "id": str(idx),
            "text": row['Review'],
            "sentiments": []
        }

        for aspect in aspects:
            sentiment = row[f"{aspect}_label"]
            if sentiment == 1:  # chỉ lấy những cái có sentiment
                aspect_sentiment_value = row[aspect]
                mapped_sent = sentiment_map.get(aspect_sentiment_value, "unknown")
                if aspect_sentiment_value != 'none':
                    entry["sentiments"].append({
                        "aspect": aspect,
                        "sentiment": mapped_sent
                    })
                else:
                    # nếu cột sentiment text bị none nhưng label == 1 thì có thể log ra kiểm tra
                    entry["sentiments"].append({
                        "aspect": aspect,
                        "sentiment": "unknown"
                    })

        result.append(entry)

    return result

In [5]:
# SETUP DATA
test_df[aspects] = test_df[aspects].fillna('none')

for aspect in aspects:
    test_df[aspect + '_label'] = (test_df[aspect] != 0).astype(int)

test_json = transform_aspect_sentiment(test_df, 0, 100)
test_json[:5]

[{'id': '0',
  'text': 'Đây là 1 trong những quán mà mình thích vì vị trà đậm và thơm cũng như mùi vị đặc trưng hơn hẳn những quán khác nè  Trà sữa trân châu sợi - 46k Trà sữa pha khá ngon, vị trà chát và mùi hương khá rõ, không quá ngọt, rất đúng với gu mình  Trà đào - 45k Vị trà đào ở đây cũng đặc biệt hơn hẳn những quán khác, không phải chua ngọt như thưởng thấy mà có mùi trà rất ngon  Cà phê đá xay - 65k Món đá xay ở đây uống cũng ngon không kém trà nè, mùi vị thơm hương cà phê, vị đắng kết hợp hoàn hảo với độ béo ngọt của whipping cream, không quá đắng, cũng không quá ngọt hay lạt lẽo mà dịu nhẹ, thơm và dễ uống lắm  Trà vải thiết quan âm - 45k Trà vải có mùi vị rất thơm ngon mùi vải mà vẫn nghe rõ vị trà, có chút vị chát nhẹ mùi trà thơm rất thích, không phải chỉ toàn vị syrup vải ngọt gắt như nhiều chỗ khác. Do trà ở đây pha khá đậm nên bạn nào uống mà đang đói sẽ dễ say nha, hoặc ban đêm có thể khó ngủ à, cảnh báo trước  Trà thiết quan âm latte - 47k Ly này thì vị trà rất đậm n

In [6]:
'''
    Nếu dùng TD-IDF thì sẽ chỉ chọn dựa vào tần suất từ, không hiểu nghĩa của câu -> bị trùng lặp
    Semantic grouping để chọn example đa dạng về nội dung, không chỉ về các từ
'''
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(test_df['Review'].tolist(), show_progress_bar=True)

# KMeans clustering (70% đa dạng, 30% khó)
n_total = 5
n_diverse = int(n_total * 0.7)
n_hard = n_total - n_diverse

kmeans = KMeans(n_clusters=n_diverse, random_state=42, n_init=10)
kmeans.fit(embeddings)

# Lấy diverse example gần cluster centroid
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, embeddings)
diverse_examples = test_df.iloc[closest]

print(f"Selected {len(diverse_examples)} diverse examples:")
for i, row in diverse_examples.iterrows():
    print(f"\nExample {i}:\n{row['Review']}")

Batches: 100%|██████████| 16/16 [00:12<00:00,  1.29it/s]


Selected 3 diverse examples:

Example 345:
Món ăn ở đây thì ngon, nhưng giá cả hơi cao. Không gian thì rất ok, phục vụ cũng nhiệt tình, chỉ có mấy anh giữ xe là hơi chán. Nhưng nếu ai muốn thử một lần sang chảnh thì có thể đến đây để tận hưởng.   Bánh bí đỏ: 48k  Bánh cuốn chiên XO: 40k  Gỏi sò huyết: 98k  Lưỡi vị nướng: 139k  Vịt quay Bắc Kinh: 600k

Example 74:
Hôm bữa mình đi có đi ăn bánh Flan trên con đường này luôn, chạy qua thấy chỗ bán bánh đúc nên tấp vào ăn thử. Mặt tiền hơi nhỏ, để chừng 4 chiếc xe là hông có đường vô luôn, bên trong không gian sạch sẽ có bàn ngồi thoải mái, có bình trà đá miễn phí nữa :)   Một chén chỉ có 15k thôi mà thịt thà cũng khá nhiều ngập mặt chứ không ít đâu nha. Bột bánh ăn cũng bình thường thôi, không được dẻo cho lắm, ăn hết chén này là ngán luôn đó. Nhân có thịt bằm với nấm mèo thêm hành phi thơm thơm nữa, nước mắm cũng vừa ăn có phần hơi ngọt tí mà mình thấy vậy là ngon rồi.  Hành phi giòn rụm luôn nè mấy chế ơi :D Ớt cho bạn nào thích ăn cay N

In [8]:
all_indices = np.arange(len(test_df))

remaining_indices = list(set(all_indices) - set(closest))

# Chọn ngẫu nhiên n_hard example từ phần còn lại 
hard_examples = test_df.iloc[remaining_indices].sample(n=n_hard, random_state=42)

print(f"Selected {len(hard_examples)} hard examples:")
for i, row in hard_examples.iterrows():
    print(f"\nExample {i}:\n{row['Review']}")

Selected 2 hard examples:

Example 486:
1 phần "Gà lên mâm" to cho 4 người gồm có: Gà ngũ vị, Gà xá xíu, Gà lá chanh, Gà muối ớt, Xôi nước cốt gà, Gỏi gà, Trứng non & Mề gà nước mắm. Mâm gà lúc dọn lên nhìn hấp dẫn và thơm lừng, nhưng mà đến lúc ăn thì thất vọng vô cùng. Xôi gà ko đậm mùi nước luộc gà, thịt gà món nào cũng khô, cứng và mặn, chỉ có gà lá chanh là tạm được. Gỏi gà là món tệ nhất vì nước trộn quá ngọt, rồi trộn xà lách chung với gỏi ăn nó cứ kỳ kỳ sao á. Phải chi gỏi chua hơn và trộn với bắp cải ko thôi là ngon rồi. Trứng non khô và nguội, mề gà ko gì đặc biệt. Nói chung trên mâm ko có món nào ngon, mấy dĩa sụn gà kêu riêng ăn còn được hơn nhiều. Ko biết mấy chỗ gà lên mâm khác thì sao chứ ko hiểu tại sao chỗ này đông dữ vậy luôn. Hình như ăn theo phong trào thôi hay sao á.

Example 73:
Hồi trước mình hay ăn bánh đúc Bà Già mà giờ nghỉ rồi, không biết có dời đi nơi khác không nữa, chỗ này mới mở sau này, cũng hay chạy qua đây nên ăn thử luôn cho biết.   Bánh đúc nóng có c

In [9]:
def select_few_shot_examples(df, text_column, n_total=5, model_name='all-MiniLM-L6-v2', random_state=42):
    n_diverse = int(n_total * 0.7)
    n_hard = n_total - n_diverse

    model = SentenceTransformer(model_name)
    embeddings = model.encode(df[text_column].tolist(), show_progress_bar=True)

    kmeans = KMeans(n_clusters=n_diverse, random_state=random_state, n_init=10)
    kmeans.fit(embeddings)

    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, embeddings)
    diverse_df = df.iloc[closest]

    all_indices = np.arange(len(df))
    remaining_indices = list(set(all_indices) - set(closest))
    hard_df = df.iloc[remaining_indices].sample(n=n_hard, random_state=random_state)

    return diverse_df.reset_index(drop=True), hard_df.reset_index(drop=True)

# SET-UP FEW-SHOT EXAMPLES
diverse, hard = select_few_shot_examples(test_df, text_column='Review', n_total=5)

few_shot_json =  transform_aspect_sentiment(diverse, 0, 100) + transform_aspect_sentiment(hard, 0, 100)


Batches: 100%|██████████| 16/16 [00:12<00:00,  1.29it/s]


In [10]:
few_shot_json[:10]

[{'id': '0',
  'text': 'Món ăn ở đây thì ngon, nhưng giá cả hơi cao. Không gian thì rất ok, phục vụ cũng nhiệt tình, chỉ có mấy anh giữ xe là hơi chán. Nhưng nếu ai muốn thử một lần sang chảnh thì có thể đến đây để tận hưởng.   Bánh bí đỏ: 48k  Bánh cuốn chiên XO: 40k  Gỏi sò huyết: 98k  Lưỡi vị nướng: 139k  Vịt quay Bắc Kinh: 600k',
  'sentiments': [{'aspect': 'AMBIENCE#GENERAL', 'sentiment': 'positive'},
   {'aspect': 'FOOD#PRICES', 'sentiment': 'negative'},
   {'aspect': 'FOOD#QUALITY', 'sentiment': 'positive'},
   {'aspect': 'SERVICE#GENERAL', 'sentiment': 'neutral'}]},
 {'id': '1',
  'text': 'Hôm bữa mình đi có đi ăn bánh Flan trên con đường này luôn, chạy qua thấy chỗ bán bánh đúc nên tấp vào ăn thử. Mặt tiền hơi nhỏ, để chừng 4 chiếc xe là hông có đường vô luôn, bên trong không gian sạch sẽ có bàn ngồi thoải mái, có bình trà đá miễn phí nữa :)   Một chén chỉ có 15k thôi mà thịt thà cũng khá nhiều ngập mặt chứ không ít đâu nha. Bột bánh ăn cũng bình thường thôi, không được dẻo ch

In [11]:
gpt_functions = [
    {
        "name": "extract_aspect_sentiment",
        "description": "Extract aspects and sentiments from text",
        "parameters": {
            "type": "object",
            "properties": {
                "results": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "aspect": {
                                "type": "string",
                                "enum": aspects
                            },
                            "sentiment": {
                                "type": "string",
                                "enum": ["positive", "negative", "neutral"]
                            }
                        },
                        "required": ["aspect", "sentiment"]
                    }
                }
            },
            "required": ["results"]
        }
    }
]

In [12]:
def evaluate_aspect_sentiment(ground_truth, predictions):
    true_aspects = []
    pred_aspects = []

    true_aspect_sentiments = []
    pred_aspect_sentiments = []

    for gt_entry, pred_entry in zip(ground_truth, predictions):
        gt_sents = gt_entry['sentiments']
        gt_aspect_set = set()
        gt_aspect_sentiment_set = set()

        for item in gt_sents:
            gt_aspect_set.add(item['aspect'])
            gt_aspect_sentiment_set.add((item['aspect'], item['sentiment']))

        true_aspects.append(gt_aspect_set)
        true_aspect_sentiments.append(gt_aspect_sentiment_set)

        pred_sents = pred_entry['results']
        pred_aspect_set = set()
        pred_aspect_sentiment_set = set()

        for item in pred_sents:
            pred_aspect_set.add(item['aspect'])
            pred_aspect_sentiment_set.add((item['aspect'], item['sentiment']))

        pred_aspects.append(pred_aspect_set)
        pred_aspect_sentiments.append(pred_aspect_sentiment_set)

    all_true_aspects = set.union(*true_aspects) if true_aspects else set()
    all_pred_aspects = set.union(*pred_aspects) if pred_aspects else set()

    tp_aspect = sum(len(gt & pred) for gt, pred in zip(true_aspects, pred_aspects))
    fp_aspect = sum(len(pred - gt) for gt, pred in zip(true_aspects, pred_aspects))
    fn_aspect = sum(len(gt - pred) for gt, pred in zip(true_aspects, pred_aspects))

    precision_aspect = tp_aspect / (tp_aspect + fp_aspect + 1e-8)
    recall_aspect = tp_aspect / (tp_aspect + fn_aspect + 1e-8)
    f1_aspect = 2 * precision_aspect * recall_aspect / (precision_aspect + recall_aspect + 1e-8)

    tp_sentiment = sum(len(gt & pred) for gt, pred in zip(true_aspect_sentiments, pred_aspect_sentiments))
    fp_sentiment = sum(len(pred - gt) for gt, pred in zip(true_aspect_sentiments, pred_aspect_sentiments))
    fn_sentiment = sum(len(gt - pred) for gt, pred in zip(true_aspect_sentiments, pred_aspect_sentiments))

    precision_sentiment = tp_sentiment / (tp_sentiment + fp_sentiment + 1e-8)
    recall_sentiment = tp_sentiment / (tp_sentiment + fn_sentiment + 1e-8)
    f1_sentiment = 2 * precision_sentiment * recall_sentiment / (precision_sentiment + recall_sentiment + 1e-8)

    return {
        "Aspect Detection F1": f1_aspect,
        "Sentiment Classification F1": f1_sentiment
    }


In [13]:
client = OpenAI()
predictions = []

# Tạo few-shot messages
few_shot_messages = []
for ex in few_shot_json:
    few_shot_messages.append({
        "role": "user",
        "content": f"Extract aspects and sentiments from the following review:\n{ex['text']}"
    })
    few_shot_messages.append({
        "role": "function",
        "name": "extract_aspect_sentiment",
        "content": json.dumps({"results": ex["sentiments"]})
    })

# Dự đoán
for data in tqdm(test_json):
    messages = [
        {"role": "system", "content": "You are an AI assistant that extracts aspects and their sentiments from text."},
        *few_shot_messages,
        {"role": "user", "content": f"Extract aspects and sentiments from the following review:\n{data['text']}"}
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        functions=gpt_functions,
        function_call={"name": "extract_aspect_sentiment"},
        temperature=0
    )

    output = response.choices[0].message.function_call.arguments
    parsed_output = json.loads(output)
    predictions.append(parsed_output)

100%|██████████| 100/100 [07:27<00:00,  4.47s/it]


In [14]:
scores = evaluate_aspect_sentiment(test_json, predictions)
print(scores)

{'Aspect Detection F1': 0.7576099160858651, 'Sentiment Classification F1': 0.665916755409106}


In [15]:
result_file = path.join(RESULT_DIR, 'ViABSA_BP_Few-shot-Restaurant-OpenAi.json')
with open(result_file, 'w') as f:
    json.dump(predictions, f, indent=4, ensure_ascii=False)